In [1]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime
import json

## Collect the conversion relation between bcr_patient_uuid and bcr_sample_barcode

In [2]:
uuid2barcode = {} # store the conversion in dict
path = "../data/biomed_clinic_data/01_csv_data"
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    df = pd.read_csv(f, sep=",")
    if "bcr_patient_uuid" in df and "bcr_patient_barcode" in df:
        for i in range(2, df.shape[0]):
            if df["bcr_patient_uuid"][i] not in uuid2barcode:
                uuid2barcode[df["bcr_patient_uuid"][i]] = (
                    df["bcr_patient_barcode"][i] + "-01"
                )

In [3]:
# store the conversion if wanted
with open("uuid2barcode.json", "w") as file:
    json.dump(uuid2barcode, file)

## Merge the original multi-omics and biomed-clinical data using bcr_sample_barcode as index

In [4]:
def merge_omics_biomed(omic_df, biomed_df, uuid2barcode, merged_name):
    (num_patients, num_features) = omic_df.shape
    print(
        f"Number of features: {num_features} \t Number of patients: {num_patients} in omics data"
    )

    (num_patients, num_features) = biomed_df.shape
    print(
        f"Number of features: {num_features} \t Number of patients: {num_patients} in biomed data"
    )

    # change uuid to barcode
    biomed_df["bcr_patient_barcode"] = "TEST"
    for i in range(biomed_df.shape[0]):
        if biomed_df["bcr_patient_uuid"][i] in uuid2barcode:
            biomed_df["bcr_patient_barcode"][i] = uuid2barcode[
                biomed_df["bcr_patient_uuid"][i]
            ]
        else:
            biomed_df["bcr_patient_barcode"][i] = None
    biomed_df.drop(columns=["bcr_patient_uuid"], inplace=True)

    # Temporary only take the "RECURRENCE" data
    target_df = biomed_df
    target_df = target_df.drop_duplicates()
    target_df = target_df.set_index("bcr_patient_barcode")

    (num_patients, num_features) = target_df.shape
    print(
        f"Number of features: {num_features} \t Number of patients: {num_patients} in omics data"
    )

    (num_patients, num_features) = omic_df.shape
    print(
        f"Number of features: {num_features} \t Number of patients: {num_patients} in biomed data"
    )
    
    merged_df = pd.merge(target_df, omic_df, left_index=True, right_index=True)
    
    # move "RECURRENCE" to the last column
    column_to_reorder = merged_df.pop("RECURRENCE")
    merged_df.insert(len(merged_df.columns), "RECURRENCE", column_to_reorder)

    (num_patients, num_features) = merged_df.shape
    print(
        f"Number of features: {num_features} \t Number of patients: {num_patients} in biomed data"
    )
    
    merged_df.to_csv("./{}".format(merged_name))

    return target_df, omic_df, merged_df

In [ ]:
# Load multi-omics data into dataframe
omics = "cnv_methyl_mrna"
omic_df = pd.read_csv(
        "../data/omics_data/1_csv_data/{}.csv".format(omics), index_col=0
    ).T
omic_df = omic_df.astype("float32")

# Load biomed-clinical data into dataframe
biomed = "biomed_clinical_85features_w_barcodes"
biomed_df = (
    pd.read_csv("../data/biomed_clinic_data/02_combined_data/{}.csv".format(biomed), index_col=0,)
    .drop_duplicates()
    .drop(columns=["bcr_sample_barcode"])
    .reset_index()
)

# Load the conversion between uuid and barcode
with open("uuid2barcode.json", "r") as file:
    uuid2barcode = json.load(file)
    
# merge multi-omics and biomed-clinical data
merged_name = "{}_biomed_clinical_85_features.csv".format(omics)
biomed_df, omic_df, merged_df = merge_omics_biomed(omic_df, biomed_df, uuid2barcode, merged_name)

Number of features: 58512 	 Number of patients: 555 in omics data
Number of features: 85 	 Number of patients: 956 in biomed data
Number of features: 84 	 Number of patients: 467 in omics data
Number of features: 58512 	 Number of patients: 555 in biomed data
Number of features: 58596 	 Number of patients: 450 in biomed data


/home/huangcancan/softwares/miniconda3/envs/dope/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
biomed_df.head()

In [ ]:
omic_df.head()

In [ ]:
merged_df.head()

## Merge the encoded omics data with the biomed-clinical data using bcr_patient_barcode as index

In [ ]:
# Load multi-omics data into dataframe
omics = "cnv_methyl_mrna"
omic_df = pd.read_csv(
        "../data/omics_data/1_csv_data/{}.csv".format(omics), index_col=0
    ).T
omic_df = omic_df.astype("float32")

# Load biomed-clinical data into dataframe
biomed = "85_features_w_barcodes"
biomed_df = (
    pd.read_csv("../data/biomed_clinic_data/02_combined_data/{}.csv".format(biomed), index_col=0,)
    .drop_duplicates()
    .drop(columns=["bcr_sample_barcode"])
    .reset_index()
)

# Load the conversion between uuid and barcode
with open("uuid2barcode.json", "r") as file:
    uuid2barcode = json.load(file)
    
# merge multi-omics and biomed-clinical data
merged_name = "{}_biomed_clinical_85_features.csv".format(omics)
merged_df = merge_omics_biomed(omic_df, biomed_df, uuid2barcode, merged_name)